In [1]:
import sys
sys.path.insert(1, '../src')
from toolboxes import Configurator, AnkiCommunicator, AnkiCardWriter
from toolboxes import StackOrganizer
from ExerciseWriter import ExerciseWriter
from Exercise import ExerciseFactory
from VocabNotes import VocabNotes
import pyperclip
from pathlib import Path
import shutil
import json


def copy_folder(source_folder, destination_folder) -> None:
    destination_path = Path(destination_folder)
    source_path = Path(source_folder)
    if not destination_path.exists():
        shutil.copytree(source_path, destination_path)

with open('../src/paths.json') as f:
    paths = json.load(f)

### Initial set-up

In [2]:
# User inputs
date = 20010101


# Make a new folder
template_folder = '../src/resources/Templates'
export_folder = f'./Exports/{date}'
copy_folder(template_folder, export_folder)

# Paths of the templates
def_template_path = '/Definition/template.tex'
collocation_template_path = '/Collocation/template.tex'
reading_template_path = '/Reading/template.tex'
gap_filling_template_path = '/Fill-in-the-gap/template.tex'
translation_template_path = '/Translation/template.tex'
comprehension_template_path = '/Comprehension/template.tex'
spelling_template_path = '/Spelling/template.tex'
correction_template_path = '/Correction/template.tex'

# Paths of the output folder
def_output_folder = export_folder + '/Definition'
collocation_output_folder = export_folder + '/Collocation'
reading_output_folder = export_folder + '/Reading'
gap_filling_output_folder = export_folder + '/Fill-in-the-gap'
translation_output_folder = export_folder + '/Translation'
spelling_output_folder = export_folder + '/Spelling'
correction_output_folder = export_folder + '/Correction'

# Creation of writers
factory = ExerciseFactory()
def_writer = ExerciseWriter(template_path=def_template_path, output_folder=def_output_folder)
collocation_writer = ExerciseWriter(template_path=collocation_template_path, output_folder=collocation_output_folder)
reading_exercise_writer = ExerciseWriter(template_path=reading_template_path, output_folder=reading_output_folder)
gap_filling_exercise_writer = ExerciseWriter(template_path=gap_filling_template_path, output_folder=gap_filling_output_folder)
translation_exercise_writer = ExerciseWriter(template_path=translation_template_path, output_folder=translation_output_folder)
spelling_exercise_writer = ExerciseWriter(template_path=spelling_template_path, output_folder=spelling_output_folder)
correction_exercise_writer = ExerciseWriter(template_path=correction_template_path, output_folder=correction_output_folder)

### Configuration for `Vocab Explorer`

In [3]:
category = 'Vocab Builder'
num_of_words_to_learn = 10

# Fetch unlearned words
configurator = Configurator(json_path=paths[category]['schedule'])
tomorrow_new = configurator.get_n_words_to_learn(num_of_words_to_learn)
with open(paths[category]['cards']) as f:
    whole_stack = json.load(f)
new_stack = {card_id: whole_stack[card_id] for card_id in tomorrow_new}
writer = AnkiCardWriter(new_stack)
Path(f'Exports/{date}_{category}').mkdir(parents=True, exist_ok=True)
writer.write_cards(f'Exports/{date}_{category}/Anki Cards.csv')
print('Go to Anki and import the cards from the file Anki Cards.csv')
configurator.study_n_words(num_of_words_to_learn)

# Fetch learned words
anki = AnkiCommunicator()
tomorrow_review = anki.get_words_for_tomorrow(deck_name=category)
review_stack = {card_id: whole_stack[card_id] for card_id in tomorrow_review}
# combine the new and review stacks
tomorrow_stack = {**new_stack, **review_stack}
print(f'''
Number of new words: {len(tomorrow_new)};
Number of old words: {len(tomorrow_review)}.
''')

Go to Anki and import the cards from the file Anki Cards.csv

Number of new words: 10;
Number of old words: 0.



### Configuration for `Everyday English`

In [ ]:
category = 'Everyday English'
num_of_words_to_learn = 0

# Fetch unlearned words
configurator = Configurator(json_path=paths[category]['schedule'])
tomorrow_new = configurator.get_n_words_to_learn(num_of_words_to_learn)
with open(paths[category]['stack']) as f:
    whole_stack = json.load(f)
new_stack = {card_id: whole_stack[card_id] for card_id in tomorrow_new}
writer = AnkiCardWriter(new_stack)
Path(f'Exports/{date}_{category}').mkdir(parents=True, exist_ok=True)
writer.write_cards(f'Exports/{date}_{category}/Anki Cards.csv')
print('Go to Anki and import the cards from the file Anki Cards.csv')
configurator.study_n_words(num_of_words_to_learn)

# Fetch learned words
anki = AnkiCommunicator()
tomorrow_review = anki.get_words_for_tomorrow(deck_name=category)
# tomorrow_review = anki.get_words_for_today(deck_name=category)
review_stack = {card_id: whole_stack[card_id] for card_id in tomorrow_review}
print(f'''
Number of new words: {len(tomorrow_new)};
Number of old words: {len(tomorrow_review)}.
''')

### 1. Definition

#### Set the word list and the set index here and get the prompt to create definitions

In [ ]:
def_start_idx = 1
def_end_idx = 10
def_set_index = 1


cropped_list = tomorrow_new[def_start_idx-1:def_end_idx]
cropped_stack = dict()
for card_id in cropped_list:
    cropped_stack[card_id] = tomorrow_stack[card_id]
stack_organizer_for_notes = StackOrganizer(stack=cropped_stack)
input_stack = stack_organizer_for_notes.reorganize()

vocab_notes = VocabNotes(word_entries=input_stack)
def_writer.render_template(exercise=vocab_notes, set_index=def_set_index)

### 2. Fill in the gaps

#### Set the word list here and get the prompt to create example sentences

In [6]:
gap_filling_start_idx = 1
gap_filling_end_idx = 10
gap_filling_set_index = 1

cropped_list = tomorrow_new[gap_filling_start_idx-1:gap_filling_end_idx]
# cropped_list = tomorrow_review[gap_filling_start_idx-1:gap_filling_end_idx]
cropped_stack = dict()
for card_id in cropped_list:
    cropped_stack[card_id] = tomorrow_stack[card_id]
stack_organizer_for_gap_filling = StackOrganizer(stack=cropped_stack, only_single_word=True)
input_stack = stack_organizer_for_gap_filling.reorganize()
gap_filling_exercise = factory.create_exercise(exercise_type='Fill in the gap', word_entries=input_stack)
gap_filling_exercise.get_prompt()

#### Paste the example sentences here

In [ ]:
gap_filling_sentences = pyperclip.paste()
gap_filling_exercise.generate_exercise(text=gap_filling_sentences)
gap_filling_exercise_writer.render_template(exercise=gap_filling_exercise, set_index=gap_filling_set_index)

### 2. Sentence Correction

#### Enter the set index and the word list here

In [ ]:
correction_start_idx = 1
correction_end_idx = 10
correction_set_index = 1

cropped_list = tomorrow_new[correction_start_idx-1:correction_end_idx]
cropped_stack = dict()
for card_id in cropped_list:
    cropped_stack[card_id] = tomorrow_stack[card_id]
stack_organizer_for_correction = StackOrganizer(stack=cropped_stack)
input_stack = stack_organizer_for_correction.reorganize()
correction_exercise = factory.create_exercise(exercise_type='Correction', word_entries=input_stack)
correction_exercise.get_prompt()

#### Paste the exercise here

In [ ]:
correction_sentences = pyperclip.paste()
correction_exercise.generate_exercise(text=correction_sentences)
correction_exercise_writer.render_template(exercise=correction_exercise, set_index=correction_set_index)

### 3. Multiple Choice - Collocations

In [4]:
collocation_start_idx = 1
collocation_end_idx = 10
collocation_set_index = 1


cropped_list = tomorrow_new[collocation_start_idx-1:collocation_end_idx]
# cropped_list = tomorrow_review[collocation_start_idx-1:collocation_end_idx]
cropped_stack = dict()
for card_id in cropped_list:
    cropped_stack[card_id] = tomorrow_stack[card_id]
stack_organizer_for_collocation = StackOrganizer(stack=cropped_stack)
input_stack = stack_organizer_for_collocation.reorganize()

collocation_exercise = factory.create_exercise(exercise_type='Collocation multiple choice', word_entries=input_stack)
collocation_exercise.get_prompt()

In [5]:
collocation_exercise_text = pyperclip.paste()
collocation_exercise.generate_exercise(text=collocation_exercise_text)
collocation_writer.render_template(exercise=collocation_exercise, set_index=collocation_set_index)

File written to Exports/20010101/Collocation/output_1.tex


### 4. Multiple Choice - Spelling

In [ ]:
spelling_start_idx = 1
spelling_end_idx = 10
spelling_set_index = 1

cropped_list = tomorrow_new[spelling_start_idx-1:spelling_end_idx]
# cropped_list = tomorrow_review[spelling_start_idx-1:spelling_end_idx]
cropped_stack = dict()
for card_id in cropped_list:
    cropped_stack[card_id] = tomorrow_stack[card_id]
stack_organizer_for_spelling = StackOrganizer(stack=cropped_stack, only_single_word=True)
input_stack = stack_organizer_for_spelling.reorganize()

spelling_exercise = factory.create_exercise(exercise_type='Spelling multiple choice', word_entries=input_stack)
spelling_exercise.get_prompt()

In [ ]:
spelling_exercise_text = pyperclip.paste()
spelling_exercise.generate_exercise(text=spelling_exercise_text)
spelling_exercise_writer.render_template(exercise=spelling_exercise, set_index=spelling_set_index)

### 5. Translation

In [ ]:
translation_start_idx = 1
translation_end_idx = 10
translation_set_index = 1

cropped_list = tomorrow_new[translation_start_idx-1:translation_end_idx]
# cropped_list = tomorrow_review[translation_start_idx-1:translation_end_idx]
cropped_stack = dict()
for card_id in cropped_list:
    cropped_stack[card_id] = tomorrow_stack[card_id]
stack_organizer_for_translation = StackOrganizer(stack=cropped_stack)
input_stack = stack_organizer_for_translation.reorganize()
translation_exercise = factory.create_exercise(exercise_type='Translation', word_entries=input_stack)
translation_exercise.generate_exercise()
translation_exercise_writer.render_template(exercise=translation_exercise, set_index=translation_set_index)

### 7. Reading

#### Enter the set index and the word list here

In [ ]:
reading_start_idx = 1
reading_end_idx = 10
reading_set_index = 1
reading_word_list = tomorrow_review[reading_start_idx-1:reading_end_idx]
reading_exercise = factory.create_exercise(exercise_type='Reading', word_entries={term: tomorrow_review_with_def[term] for term in reading_word_list})
reading_exercise.get_prompt()

#### Paste the passages here

In [ ]:
passages = pyperclip.paste()
reading_exercise.import_passage(text=passages)
reading_exercise.get_second_prompt()

#### Paste the cloze exercise here

In [ ]:
cloze_exercise_text = pyperclip.paste()

reading_exercise.import_exercise(text=cloze_exercise_text)
reading_exercise.finish_import()
reading_gatherer.import_exercise(exercise=reading_exercise)
reading_exercise_writer.render_template(set_index=reading_set_index)

### 8. Dialogue

#### Enter the set index here

In [ ]:
dialogue_start_idx = 6
dialogue_end_idx = 11
dialogue_set_index = 2

# dialogue_word_list = tomorrow_new[dialogue_start_idx-1:dialogue_end_idx]
# dialogue_word_entries = {term: tomorrow_new_with_def[term] for term in dialogue_word_list}

dialogue_word_list = tomorrow_review[dialogue_start_idx-1:dialogue_end_idx]
dialogue_word_entries = {term: tomorrow_review_with_def[term] for term in dialogue_word_list}

dialogue_exercise = factory.create_exercise(exercise_type='Dialogue', word_entries={term: dialogue_word_entries[term] for term in dialogue_word_list})
dialogue_exercise.get_prompt()

In [ ]:
dialogue_dict_text = pyperclip.paste()
dialogue_dict = json.loads(dialogue_dict_text)
dialogue_exercise.generate_exercise(dialogue_dict=dialogue_dict)
dialogue_exercise.finish_import()
dialogue_gatherer.import_exercise(exercise=dialogue_exercise)
dialogue_exercise_writer.render_template(set_index=dialogue_set_index)